<a href="https://colab.research.google.com/github/harshalc108/Project-demo-pipelines/blob/main/batch_dataflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install apache-beam[gcp]

In [ ]:
from apache_beam.io.gcp.bigquery import WriteToBigQuery
import argparse
import os
from datetime import date
from google.cloud import bigquery

path1='bigquery-demo-335405-d7f1212692d0.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=path1

tab_id='batchdata.table1'
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

parser = argparse.ArgumentParser()
# parser.add_argument('--my-arg')
args, beam_args = parser.parse_known_args()

# Create and set your Pipeline Options.
beam_options = PipelineOptions(
    beam_args,
    runner='DataflowRunner',
    temp_location='gs://demo_data0/temp1999',
    service_account_email='demo-python@bigquery-demo-335405.iam.gserviceaccount.com',
    region='us-central1',
    project='bigquery-demo-335405',
    stage_location='gs://data_00/stage'
    )

#args = beam_options.view_as(MyOptions)
#today= 2548566
client = bigquery.Client()

dataset_id = "bigquery-demo-335405.batchdata"

try:
  client.get_dataset(dataset_id)

except:
  dataset = bigquery.Dataset(dataset_id)  

  dataset.location = "US"
  dataset.description = "dataset"
  dataset_ref = client.create_dataset(dataset)

def to_json(fields):
    json_str = {"Region":fields[0],
                 "Country": fields[1],
                 "Item_Type": fields[2],
                 "Sales_Channel": fields[3],
                "Order_Priority": fields[4],
                "Order_Date": fields[5],
                "Order_ID": fields[6],
                "Ship_Date": fields[7],
                "Units_Sold": fields[8],
                "Unit_Cost": fields[9],
                "Total_Profit": fields[10]

                 
                 
                 }
    return json_str
schema_1='Region:STRING,Country:STRING,Item_Type:STRING,Sales_Channel:STRING,Order_Priority:STRING,Order_Date:STRING,Order_ID:INTEGER,Ship_Date:STRING,Units_Sold:INTEGER,Unit_Cost:FLOAT,Total_Profit:FLOAT '


with beam.Pipeline(options=beam_options) as pipeline:
    data_ingestion=(
    pipeline
    | beam.io.ReadFromText('gs://data_00/sales_record.csv',skip_header_lines=1)
    | beam.Map(lambda x:x.split(','))
    | "filter Offline Sales_Channel">> beam.Filter(lambda x:x[3]=="Offline")
    | "convert to json2">> beam.Map(to_json)
    | "write to Offline table">> beam.io.WriteToBigQuery(
        tab_id,
        schema=schema_1,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
        write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND
    )
    )

/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery.py:2102: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  self.table_reference.projectId = pcoll.pipeline.options.view_as(
/usr/local/lib/python3.7/dist-packages/apache_beam/io/gcp/bigquery_file_loads.py:1112: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
